# **KoBigBird - Data Augmentation 적용**

**수정 사항**

*   학습 데이터(df_final)에 라벨을 유지한 채로 변형 샘플을 추가해서 학습셋을 더 풍부하게 만든 것이 핵심 수정 사항
*   공백/구두점 변형(의미 보존) : !!! 같은 과한 반복을 적당히 변형(강조 1회 정도)
*   갈취(라벨 1) : 금액 표현 다양화
*   협박(라벨 0), 기타 괴롭힘(라벨 3): 라벨 고유 꼬리 문구 소량 추가
*   직장 괴롭힘(라벨 2) : 역할(직책) 표현 치환
*   일반(라벨 4) : 최소 적용(라벨 혼입(유해 표현 섞임) 위험이 있어 label-specific 증강을 거의 안 하고 비율도 낮게 설정)

*   역번역(Back-Translation, ko→en→ko) 기반 증강 추가 : 라벨별 적용 가능(기본은 유해 클래스만)


In [1]:
import pandas as pd
import numpy as np
import torch
import random
import os
import sys

# ==========================================
# 1. 라이브러리 및 환경 설정
# ==========================================
try:
    from sklearn.metrics import classification_report, accuracy_score, f1_score
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, BigBirdConfig
    from datasets import Dataset
    from sklearn.model_selection import train_test_split
except ImportError:
    sys.exit("❌ 라이브러리가 설치되지 않았습니다. !pip install transformers datasets accelerate scikit-learn 실행 필요")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 [KoBigBird] 사용 장치: {device}")

# ==========================================
# 2. 데이터 로드 및 병합
# ==========================================
url = "https://raw.githubusercontent.com/tunib-ai/DKTC/main/data/train.csv"
try:
    df_threat = pd.read_csv(url)[['class', 'conversation']]
    print(f"✅ 위협 데이터 로드 완료: {len(df_threat)}개")
except Exception as e:
    sys.exit(f"❌ 위협 데이터 로드 실패: {e}")

# 일반 대화 파일 로드 (파일명 확인)
normal_file = "normal_conversation (1).csv"
if not os.path.exists(normal_file):
    if os.path.exists("normal_conversation.csv"):
        normal_file = "normal_conversation.csv"
    else:
        sys.exit(f"❌ '{normal_file}' 파일을 찾을 수 없습니다.")

print(f"📂 사용할 일반 대화 파일: {normal_file}")
df_normal = pd.read_csv(normal_file)
df_normal['conversation'] = df_normal['conversation'].str.replace(r'(^|\n)[AB]:\s*', '', regex=True)
if 'class' not in df_normal.columns:
    df_normal['class'] = '일반 대화'
df_normal = df_normal[['class', 'conversation']]
print(f"✅ 일반 대화 데이터 로드 완료: {len(df_normal)}개")

# 병합
df_final = pd.concat([df_threat, df_normal], ignore_index=True)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# ==========================================
# 3.1. 라벨 인코딩 & 데이터셋 변환
# ==========================================
label_map = {
    '협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3,
    '협박': 0, '갈취': 1, '직장 내 괴롭힘': 2, '기타 괴롭힘': 3,
    '일반 대화': 4
}
df_final['label'] = df_final['class'].map(label_map)
df_final = df_final.dropna(subset=['label'])
df_final['label'] = df_final['label'].astype(int)


# ==========================================
# 3.2. 클래스별 Augmentation (라벨 보존)
# ==========================================
# 현재 형태(데이터 로드→라벨링→split→학습)는 그대로 유지하면서,
# train/val split 이전에 라벨별 비율로 증강 추가
AUGMENT = True
AUG_SEED = 42

# 라벨별 증강 비율(원본 대비 추가 생성 비중)
# - 유해(0~3)는 표현 다양성 확보를 위해 더 크게 적용
# - 일반(4)은 의미 훼손이나 혼입 위험 때문에 아주 보수적으로 적용
AUG_RATIO = {0: 0.6, 1: 0.6, 2: 0.6, 3: 0.4, 4: 0.1}

import re

_money_re = re.compile(r"(\\d{1,3}(?:,\\d{3})+|\\d+)(\\s*)(원|만원|천원)?")
_space_re = re.compile(r"\\s+")
_punc_re = re.compile(r"([!?.,])\\1{1,}")

def _normalize(text: str) -> str:
    text = str(text)
    text = text.replace("\u200b", " ").replace("\xa0", " ")
    text = _space_re.sub(" ", text).strip()
    return text

def _aug_spacing(text: str, rng: random.Random) -> str:
    # 공백/구두점 약간 변형(의미 보존)
    t = _normalize(text)
    if rng.random() < 0.35:
        t = re.sub(r"\\s*([!?.,])\\s*", r"\\1 ", t)
    t = _space_re.sub(" ", t).strip()
    # 문장 끝 강조 1회
    if rng.random() < 0.20:
        t = _punc_re.sub(r"\\1\\1", t)
    return t

def _money_variant(t: str, rng: random.Random) -> str:
    # 갈취/돈 관련: 금액 표기 다양화 (예: 10만원 ↔ 100,000원)
    def repl(m):
        num = m.group(1).replace(",", "")
        unit = m.group(3) or ""
        try:
            v = int(num)
        except:
            return m.group(0)
        if unit == "만원":
            v_won = v * 10000
        elif unit == "천원":
            v_won = v * 1000
        else:
            v_won = v
        cand = [f"{v_won:,}원", f"{v_won}원"]
        if v_won % 10000 == 0:
            cand.append(f"{v_won//10000}만원")
        if v_won % 1000 == 0:
            cand.append(f"{v_won//1000}천원")
        return rng.choice(cand)
    return _money_re.sub(repl, t)

def _label_specific(text: str, label: int, rng: random.Random) -> str:
    t = _normalize(text)

    # 라벨별 의미를 바꾸지 않는 표현 variant를 추가 및 치환
    if label == 0:  # 협박
        tails = [" 가만 안 둔다.", " 각오해.", " 후회하게 해줄게.", " 진짜로 가만두지 않을 거야."]
        if rng.random() < 0.35:
            t = t + rng.choice(tails)
        return t

    if label == 1:  # 갈취
        t = _money_variant(t, rng)
        tails = [" 지금 보내.", " 송금해.", " 계좌로 입금해.", " 돈 보내라."]
        if rng.random() < 0.35:
            t = t + rng.choice(tails)
        return t

    if label == 2:  # 직장 괴롭힘
        role_map = {
            "팀장": ["파트장", "리더", "조장"],
            "부장": ["상사", "관리자"],
            "인사팀": ["HR", "총무팀"],
            "사장": ["대표", "회장"],
        }
        for k, vs in role_map.items():
            if k in t and rng.random() < 0.30:
                t = t.replace(k, rng.choice(vs), 1)
        return t

    if label == 3:  # 기타 괴롭힘
        tails = [" 진짜 못됐다.", " 너무 심하다.", " 그만 좀 해.", " 이건 괴롭힘이야."]
        if rng.random() < 0.25:
            t = t + rng.choice(tails)
        return t

    return t  # 일반(4) 포함: 라벨별 변형은 최소화

def augment_text(text: str, label: int, rng: random.Random) -> str:
    t = _aug_spacing(text, rng)
    # 일반은 보수적으로: label-specific는 제외(혼입 방지)
    if label != 4 and rng.random() < 0.85:
        t = _label_specific(t, label, rng)
    return _normalize(t)

def apply_classwise_augmentation(df: pd.DataFrame,
                                 text_col: str = "conversation",
                                 label_col: str = "label",
                                 ratio_by_label: dict = None,
                                 seed: int = 42) -> pd.DataFrame:
    if ratio_by_label is None:
        ratio_by_label = {0:0.6, 1:0.6, 2:0.6, 3:0.4, 4:0.1}
    rng = random.Random(seed)

    aug_rows = []
    for lab, ratio in ratio_by_label.items():
        sub = df[df[label_col] == lab]
        if len(sub) == 0 or ratio <= 0:
            continue
        n_add = int(len(sub) * ratio)
        if n_add <= 0:
            continue
        pick = sub.sample(n=n_add, replace=True, random_state=seed)
        for _, r in pick.iterrows():
            new_text = augment_text(r[text_col], int(lab), rng)
            aug_rows.append({"class": r.get("class", None), text_col: new_text, label_col: int(lab)})

    if not aug_rows:
        return df

    df_aug = pd.concat([df, pd.DataFrame(aug_rows)], ignore_index=True)
    df_aug = df_aug.sample(frac=1, random_state=seed).reset_index(drop=True)
    return df_aug



# ==========================================
# 3.2. (추가) 역번역(Back-Translation) Augmentation
# ==========================================
# - ko → en → ko 로 의미를 최대한 유지하면서 표현을 다양화
# - ⚠️ 번역 품질/의미 훼손 가능성이 있어 기본값은 일반대화(4) 제외 권장
# - 실행 시 Hugging Face 번역 모델을 다운로드합니다(Colab에서 1회 캐시됨).

USE_BACK_TRANSLATION = True

# 라벨별 역번역 증강 비율(원본 대비 추가 생성 비중)
# 기본: 유해(0~3)만 적용, 일반(4)은 0
BT_RATIO = {0: 0.3, 1: 0.3, 2: 0.3, 3: 0.2, 4: 0.0}

# 번역 모델(가벼운 MarianMT 계열)
KO_EN_MODEL = "Helsinki-NLP/opus-mt-ko-en"
EN_KO_MODEL = "Helsinki-NLP/opus-mt-en-ko"

def _load_translation_models(device):
    from transformers import MarianMTModel, MarianTokenizer
    ko_en_tok = MarianTokenizer.from_pretrained(KO_EN_MODEL)
    ko_en = MarianMTModel.from_pretrained(KO_EN_MODEL).to(device).eval()
    en_ko_tok = MarianTokenizer.from_pretrained(EN_KO_MODEL)
    en_ko = MarianMTModel.from_pretrained(EN_KO_MODEL).to(device).eval()
    return (ko_en_tok, ko_en, en_ko_tok, en_ko)

def _translate_batch(texts, tok, model, device, batch_size=16, max_length=256, num_beams=4):
    import torch
    outs = []
    for i in range(0, len(texts), batch_size):
        batch = [str(t) for t in texts[i:i+batch_size]]
        enc = tok(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
        with torch.no_grad():
            gen = model.generate(**enc, max_length=max_length, num_beams=num_beams)
        outs.extend(tok.batch_decode(gen, skip_special_tokens=True))
    return outs

def back_translate_texts(texts, device, batch_size=16, max_length=256, num_beams=4):
    """
    입력: 한국어 텍스트 리스트
    출력: ko→en→ko 역번역 결과 리스트(길이 동일)
    """
    try:
        ko_en_tok, ko_en, en_ko_tok, en_ko = _load_translation_models(device)
        en_texts = _translate_batch(texts, ko_en_tok, ko_en, device, batch_size=batch_size, max_length=max_length, num_beams=num_beams)
        ko_texts = _translate_batch(en_texts, en_ko_tok, en_ko, device, batch_size=batch_size, max_length=max_length, num_beams=num_beams)
        return [ _normalize(t) for t in ko_texts ]
    except Exception as e:
        # 번역 모델 로드/생성 실패 시 원문 반환 (학습 파이프라인 중단 방지)
        print(f"⚠️ [Back-Translation] 실패하여 원문을 사용합니다: {e}")
        return [ _normalize(t) for t in texts ]

def apply_back_translation(df: pd.DataFrame,
                           text_col: str = "conversation",
                           label_col: str = "label",
                           ratio_by_label: dict = None,
                           seed: int = 42,
                           device = None,
                           batch_size: int = 16,
                           max_length: int = 256,
                           num_beams: int = 4) -> pd.DataFrame:
    """
    라벨별로 일부 샘플을 뽑아 역번역한 문장을 추가.
    """
    if ratio_by_label is None:
        ratio_by_label = {0:0.3, 1:0.3, 2:0.3, 3:0.2, 4:0.0}
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    aug_rows = []
    rng = random.Random(seed)

    for lab, ratio in ratio_by_label.items():
        sub = df[df[label_col] == lab]
        if len(sub) == 0 or ratio <= 0:
            continue
        n_add = int(len(sub) * ratio)
        if n_add <= 0:
            continue

        pick = sub.sample(n=n_add, replace=True, random_state=seed)
        texts = [ _normalize(t) for t in pick[text_col].tolist() ]

        # 역번역 수행
        bt_texts = back_translate_texts(texts, device=device, batch_size=batch_size, max_length=max_length, num_beams=num_beams)

        # 너무 비슷한(효과 미미) 샘플은 일부 제거(보수적)
        for orig, new_text, (_, r) in zip(texts, bt_texts, pick.iterrows()):
            if not new_text:
                continue
            if new_text == orig:
                # 완전 동일하면 스킵
                continue
            # 길이 변화가 너무 큰 경우 스킵(의미 훼손 가능성)
            if len(new_text) < 0.5 * len(orig) or len(new_text) > 2.0 * len(orig):
                continue
            aug_rows.append({"class": r.get("class", None), text_col: new_text, label_col: int(lab)})

    if not aug_rows:
        return df

    df_bt = pd.concat([df, pd.DataFrame(aug_rows)], ignore_index=True)
    df_bt = df_bt.sample(frac=1, random_state=seed).reset_index(drop=True)
    return df_bt

if AUGMENT:
    print("\n[Augmentation] 적용 전 라벨 분포:\n", df_final['label'].value_counts().sort_index())
    df_final = apply_classwise_augmentation(df_final, ratio_by_label=AUG_RATIO, seed=AUG_SEED)
    print("\n[Augmentation] 적용 후 라벨 분포:\n", df_final['label'].value_counts().sort_index())

    if USE_BACK_TRANSLATION:
        print("\n[Back-Translation] 적용 전 라벨 분포:\n", df_final['label'].value_counts().sort_index())
        df_final = apply_back_translation(df_final, ratio_by_label=BT_RATIO, seed=AUG_SEED, device=device, batch_size=16, max_length=256, num_beams=4)
        print("\n[Back-Translation] 적용 후 라벨 분포:\n", df_final['label'].value_counts().sort_index())


train_df, val_df = train_test_split(df_final, test_size=0.2, random_state=42, stratify=df_final['label'])
train_dataset = Dataset.from_pandas(train_df[['conversation', 'label']])
val_dataset = Dataset.from_pandas(val_df[['conversation', 'label']])

# ==========================================
# 4. 모델 & 토크나이저 (KoBigBird)
# ==========================================
# 🚨 [변경됨] KoBigBird 모델 사용
MODEL_NAME = "monologg/kobigbird-bert-base"
print(f"🔄 모델 로딩 중: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 🚨 [중요] BigBird 설정: 짧은 문장(4096 토큰 미만) 처리 시 original_full 사용 권장
# 만약 메모리가 부족하면 이 부분을 삭제하여 기본값(block_sparse)으로 사용하세요.
config = BigBirdConfig.from_pretrained(MODEL_NAME)
config.attention_type = "original_full"
config.num_labels = 5

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config).to(device)

def preprocess_function(examples):
    # BigBird는 긴 문맥(최대 4096) 처리가 장점이지만,
    # 이번 데이터셋은 대화가 그렇게 길지 않으므로 512~1024 정도로 설정해도 충분합니다.
    # 여기서는 안전하게 512로 설정합니다. (필요 시 1024로 늘리세요)
    return tokenizer(examples["conversation"], truncation=True, max_length=512)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==========================================
# 5. 학습 (Training)
# ==========================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results_bigbird",  # [변경됨] 저장 폴더
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # [변경됨] 메모리 절약을 위해 8로 조정 (가능하면 16)
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,   # 배치 8 * 2 = 실제 16 효과
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("\n🦅 KoBigBird 학습 시작! (시간이 조금 더 걸릴 수 있습니다)")
trainer.train()

# ==========================================
# 6. 모델 저장
# ==========================================
SAVE_PATH = "./final_model_bigbird" # [변경됨] 저장 폴더
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"\n✅ 학습 완료! 모델이 '{SAVE_PATH}' 폴더에 저장되었습니다.")


🚀 [KoBigBird] 사용 장치: cuda
✅ 위협 데이터 로드 완료: 3950개
📂 사용할 일반 대화 파일: normal_conversation.csv
✅ 일반 대화 데이터 로드 완료: 800개

[Augmentation] 적용 전 라벨 분포:
 label
0     896
1     981
2     979
3    1094
4     800
Name: count, dtype: int64

[Augmentation] 적용 후 라벨 분포:
 label
0    1433
1    1569
2    1566
3    1531
4     880
Name: count, dtype: int64

[Back-Translation] 적용 전 라벨 분포:
 label
0    1433
1    1569
2    1566
3    1531
4     880
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


✅ 학습 완료! 모델이 './final_model_bigbird' 폴더에 저장되었습니다.


In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ==========================================
# 1. 설정 및 데이터 준비
# ==========================================
MODEL_PATH = "./final_model_bigbird"       # 학습된 모델 경로
DATA_PATH = "train.csv" # 데이터 파일 경로

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 로드
df = pd.read_csv(DATA_PATH)
df['conversation'] = df['conversation'].astype(str)

# 라벨 재매핑 (안전장치)
label_map = {
    '협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3,
    '협박': 0, '갈취': 1, '직장 내 괴롭힘': 2, '기타 괴롭힘': 3,
    '직장 괴롭힘': 2, '기타 괴롭힘': 3,
    '일반 대화': 4
}
df['label'] = df['class'].map(label_map)

# 결측치 제거
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

# ---------------------------------------------------------
# ⚖️ [핵심] 클래스별 100개씩 균형 샘플링
# ---------------------------------------------------------
# 각 라벨별로 랜덤하게 100개씩 뽑습니다.
try:
    test_df = df.groupby('label').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)
    print(f"테스트 데이터셋 구성 완료: 총 {len(test_df)}개")
    print(test_df['class'].value_counts()) # 각 100개인지 확인
except ValueError as e:
    print(f"오류: 데이터가 부족하여 클래스별 100개를 뽑을 수 없습니다. ({e})")
    # 예외 시 전체 데이터 사용
    test_df = df

# ==========================================
# 2. 모델 로드 및 예측
# ==========================================
print("\n모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (코드가 멈추지 않도록 임시 종료 처리 필요 시 exit())

# 예측 함수
def predict_batch(texts, batch_size=32):
    all_preds = []
    # 데이터가 많을 경우 배치를 나눠서 처리
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        all_preds.extend(preds)
    return np.array(all_preds)

print("예측 시작... (잠시만 기다려주세요)")
y_true = test_df['label'].tolist()
y_pred = predict_batch(test_df['conversation'].tolist())

# ==========================================
# 3. 평가 지표 출력 (Text Only)
# ==========================================
name_map = {0:'협박', 1:'갈취', 2:'직장 괴롭힘', 3:'기타 괴롭힘', 4:'일반 대화'}

print("\n" + "="*50)
print("[최종 모델 평가 리포트 (Test Set: 100 samples/class)]")
print("="*50)

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"정확도 (Accuracy): {acc:.4f}")
print(f"F1 점수 (Weighted): {f1:.4f}")
print("-" * 50)

print("\n[클래스별 상세 지표]")

present_labels = sorted(set(y_true) | set(y_pred))  # ★ 여기 추가
print(classification_report(
    y_true, y_pred,
    labels=present_labels,
    target_names=[name_map[i] for i in present_labels],
    digits=4,
    zero_division=0
))

print("="*50)

테스트 데이터셋 구성 완료: 총 400개
class
협박 대화          100
갈취 대화          100
직장 내 괴롭힘 대화    100
기타 괴롭힘 대화      100
Name: count, dtype: int64

모델 로딩 중...


/tmp/ipython-input-1014563579.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = df.groupby('label').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)


Loading weights:   0%|          | 0/203 [00:00<?, ?it/s]

예측 시작... (잠시만 기다려주세요)

[최종 모델 평가 리포트 (Test Set: 100 samples/class)]
정확도 (Accuracy): 0.9700
F1 점수 (Weighted): 0.9699
--------------------------------------------------

[클래스별 상세 지표]
              precision    recall  f1-score   support

          협박     0.9798    0.9700    0.9749       100
          갈취     0.9423    0.9800    0.9608       100
      직장 괴롭힘     0.9901    1.0000    0.9950       100
      기타 괴롭힘     0.9688    0.9300    0.9490       100

    accuracy                         0.9700       400
   macro avg     0.9702    0.9700    0.9699       400
weighted avg     0.9702    0.9700    0.9699       400



In [3]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# =========================================================
# Kaggle 제출용 submission.csv 생성 (sample_submission 기준)
# =========================================================
# ✅ Kaggle에서는 Data 탭에 있는 sample_submission.csv / test.csv를 그대로 사용해야 합니다.
# - 행 수: sample_submission과 동일
# - 컬럼: sample_submission과 동일(보통 idx, class)
# - idx 정렬: sample_submission의 idx 순서를 그대로 유지

MODEL_PATH = "./final_model_bigbird"  # 학습된 모델 폴더(working 디렉토리에 있어야 함)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 장치:", device)

# -------------------------
# 1) Kaggle 입력 파일 찾기
# -------------------------
BASE_DIR = "/kaggle/input"
sample_path, test_path = None, None

if os.path.exists(BASE_DIR):
    for root, _, files in os.walk(BASE_DIR):
        for f in files:
            if f == "sample_submission.csv":
                sample_path = os.path.join(root, f)
            elif f == "test.csv":
                test_path = os.path.join(root, f)

# Kaggle이 아닌 환경(Colab 등) 대비: 현재 폴더도 탐색
if sample_path is None and os.path.exists("sample_submission.csv"):
    sample_path = "sample_submission.csv"
if test_path is None and os.path.exists("test.csv"):
    test_path = "test.csv"

print("sample_submission.csv:", sample_path)
print("test.csv:", test_path)

if sample_path is None or test_path is None:
    raise FileNotFoundError(
        "sample_submission.csv 또는 test.csv를 찾지 못했습니다. "
        "Kaggle에서는 Data 탭을 추가했는지, Colab에서는 파일을 업로드했는지 확인하세요."
    )

sample = pd.read_csv(sample_path)
test = pd.read_csv(test_path)

print("sample shape:", sample.shape, "columns:", sample.columns.tolist())
print("test shape:", test.shape, "columns:", test.columns.tolist())

# -------------------------
# 2) 모델 로드
# -------------------------
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(
        f"MODEL_PATH를 찾지 못했습니다: {MODEL_PATH}\n"
        "Kaggle에서는 학습된 모델 폴더(final_model_bigbird)를 Dataset으로 업로드하거나, "
        "노트북에서 학습 후 /kaggle/working 아래에 저장했는지 확인하세요."
    )

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
model.eval()

# -------------------------
# 3) 예측
# -------------------------
# test 텍스트 컬럼 자동 선택
if "conversation" in test.columns:
    text_col = "conversation"
elif "text" in test.columns:
    text_col = "text"
else:
    raise KeyError(f"test.csv에서 텍스트 컬럼을 찾지 못했습니다. 현재 컬럼: {test.columns.tolist()}")

def predict_batch(texts, batch_size=32, max_length=256):
    preds = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_length
        ).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        preds.extend(torch.argmax(logits, dim=-1).cpu().numpy().tolist())
    return np.array(preds, dtype=int)

preds = predict_batch(test[text_col].astype(str).tolist(), batch_size=32, max_length=256)
print("preds len:", len(preds), "unique labels:", sorted(set(preds.tolist())))

# -------------------------
# 4) 제출 파일 생성 (sample_submission 틀 사용)
# -------------------------
sub = sample.copy()

# idx로 정렬/매핑 (가장 안전)
if "idx" in sub.columns and "idx" in test.columns:
    pred_map = dict(zip(test["idx"].astype(str), preds))
    sub["class"] = sub["idx"].astype(str).map(pred_map)

    if sub["class"].isna().any():
        missing = sub[sub["class"].isna()]["idx"].head(10).tolist()
        raise ValueError(f"idx 매핑 실패로 예측 누락 발생. 예: {missing}")
else:
    # idx 컬럼이 없다면 길이 일치로만 검증
    if len(sub) != len(preds):
        raise ValueError(f"행 수 불일치: sample={len(sub)}, preds={len(preds)}")
    sub["class"] = preds

# class 타입 정리
sub["class"] = sub["class"].astype(int)

# 최종 검증: 컬럼/행 수
assert list(sub.columns) == list(sample.columns), "컬럼명이 sample_submission과 다릅니다."
assert len(sub) == len(sample), "행 수가 sample_submission과 다릅니다."

save_path = "submission.csv"
sub.to_csv(save_path, index=False)

print("\n" + "="*60)
print("✅ 제출 파일 생성 완료:", save_path)
print("shape:", sub.shape)
print("columns:", sub.columns.tolist())
print("="*60)
print(sub.head())


사용 장치: cuda
오류: 'test.json' 파일을 찾을 수 없습니다.
모델 로딩 중...


Loading weights:   0%|          | 0/203 [00:00<?, ?it/s]

예측 시작...


100%|██████████| 2/2 [00:00<00:00, 10.04it/s]


제출 파일 생성 완료: submission.csv
     idx  class
0  t_000      3
1  t_001      1
